# 라이브러리 불러오기

In [1]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.6 MB/s eta 0:00:00


In [2]:
!pip install -U albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 KB 11.8 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.2.1
    Uninstalling albumentations-1.2.1:
      Successfully uninstalled albumentations-1.2.1


In [3]:
!pip install ttach

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

import albumentations as A
from albumentations.pytorch import ToTensorV2

import ttach as tta

device = torch.device('cuda')

# 원 데이터를 불러옵니다...

In [5]:
os.chdir("/content/drive/MyDrive/DL_Project")

In [6]:
train_png = sorted(glob('train/*.png'))
test_png = sorted(glob('test/*.png'))

In [7]:
len(train_png), len(test_png)

(7225, 2154)

In [8]:
train_y = pd.read_csv("/content/drive/MyDrive/DL_Project/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

# 가중치 정규화(가 뭘까)?

In [9]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1] # 역으로 읽기기
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) # 흑백 변환
    img = cv2.resize(img, (384, 384))
    return img

In [ ]:
train_imgs = [img_load(m) for m in tqdm(train_png)] # train_imgs 다운로드
test_imgs = [img_load(n) for n in tqdm(test_png)] # test_imgs 다운로드드

100%|██████████| 2154/2154 [46:46<00:00,  1.30s/it]


# 들어가기 전 말할 것들
- 이번 optimizer는 통상적으로 adam을 쓸 거예요
- 제일 안정적으로 결과가 잘 나오는 것이 adam이라서 그렇습니다
- 그리고 efficientnet 모델 종류는 여러가지를 써 봅시다

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomCrop(356),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFilp(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


test_transform = transforms.Compose([
    transforms.ToTensor(),  # 이 과정에서 [0, 255]의 범위를 갖는 값들을 [0.0, 1.0]으로 정규화, torch.FloatTensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])   #  정규화(normalization)
    
])

In [ ]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            img = train_transform(image=img)
        if self.mode=='test':
            img = test_transform(image=img)
        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
# 여기는 score를 계산하는 것으로 보입니다.
def score_function(real, pred):
  score = f1_score(real, pred, average="macro")
  return score

# 데이터 셋과 데이터 로더
- 

In [ ]:
# dataset : 전체 dataset 구성 dataloader : mini batch 만드는 역할할
batch_size = 32 # batch_size : 사진들을 몇 개 묶음으로 할 거냐
epochs = 70 # 학습 시도 횟수

# 데이터 셋과 데이터 로더 부분

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train') # train 데이터셋 학습 모델
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test') # test 데이터셋 학습 모델
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# training

- 여기가 코딩함에 따라 결과가 바뀌는 과정 == yolov5 train.py 
- 훈련의 예시에는 여러 개가 있고 밑에 예시를 가져올게요

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score # 모델 스코어

model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 



best=0
for epoch in range(epochs): # 학습 기본 설정 setting
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)

        if len(y) != len(pred):
            continue

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s') # 시간 확인 안내내
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    if train_f1 > 0.9980:
      model_path = "/content/drive/MyDrive/DL_Project/model"
      torch.save(model.state_dict(), f"{train_f1}.pt")

IndexError: ignored

# 모델 학습

# 모델 추론
- 이건 말 그대로 모델의 학습이 끝난 직후, 어떤 방식으로 모델의 정확도를 높일 수 있을 지를 생각하는 코드입니다.
- 여기도 바꿔줘야 되요!

In [ ]:
model.eval()
f_pred = [] 

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()} 

f_result = [label_decoder[result] for result in f_pred]

# 결과창

In [ ]:
submission = pd.read_csv("open/sample_submission.csv") # 데이터 제출출

submission["label"] = f_result

submission

- 여기까지가 dacon에서 제공한 base_model입니다
- 나머지는 저희가 발전시키면 되겠죠?

# 모델 정확도 증가 방법

## 데이터 증강(data argumentation)

- 여기서 TTA란 기법이 나오며, test-time argumentation이라고 합니다. tta는 모델 학습 중에 데이터 증강을 도입하는 것이라고 보면 될 것 같아요
- 데이터 증강이란 거는 쉽게 말해서 뒤집고 돌리고 한다고 보시면 됩니다.

In [ ]:
!pip install albumentation==0.4.6

In [ ]:
# 하는 방법(예시)
import albumentations
import albumentations.pytorch

aug = albumentations.Compose([
      albumentations.Resize(224, 224),
      albumentations.HorizontalFlip(), #수평 뒤집기
      albumentations.VerticalFlip(), #수직 뒤집기
      albumentations.OneOf([
                          albumentations.Rotate(), # 돌리기기
                          albumentations.ShiftScaleRotate()
 
      ], p=1),
      albumentations.augmentations.transforms.Normalize(mean=(0.5,), std=(0.5,), p=1.0), #정규화화
      albumentations.pytorch.transforms.ToTensorV2(p=1.0)
      ])
aug2 = albumentations.Compose([
      albumentations.Resize(224, 224),
      albumentations.Rotate(), # 돌리기기
      albumentations.augmentations.transforms.Normalize(mean=(0.5,), std=(0.5,), p=1.0),
      albumentations.pytorch.transforms.ToTensorV2(p=1.0)
      ])

- 이미지 기법에 대한 더 자세한 설명은 강사님이 주신 자료 중 7-1. image-processing의 동명의 파일 
- image-processing를 보면 더 자세히 나와 있어요.

## 앙상블 기법
- 여러분이 아시는 그 앙상블로, 파라미터나 epoch, batch, resize 등 다양한 기법으로 만들어진 모델 중 
- 괜찮은 것들을 찾아 앙상블하는 것입니다.

# 학습(다른 예시)
- 1번째 거는 다른 팀들이 학습 코드를 짠 것들이에요


In [ ]:
# 시도 횟수
n_epochs = 100
valid_loss_min = np.inf # 100보다 작을 시 멈춤

# keep track of training and validation loss
train_loss = torch.zeros(n_epochs)
valid_loss = torch.zeros(n_epochs)

train_F1 = torch.zeros(n_epochs)
valid_F1 = torch.zeros(n_epochs)
model.to(device)

for e in range(0, n_epochs):

   
    ###################
    # 모델 학습       #
    ###################
    model.train()
    for data, labels in tqdm(train_dataloader):
        # move tensors to GPU if CUDA is available
        data, labels = data.to(device), labels.to(device)
        # 순전파 :compute predicted outputs by passing inputs to the model
        logits = model(data)
        # 배치의 손실율 계산
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        # 역전파 : compute gradient of the loss with respect to model parameters
        loss.backward()
        # 싱글 옵티미제이션(adam)
        optimizer.step()
        # 학습 손실율 업데이트
        train_loss[e] += loss.item()
        # 학습 스코어 계산산
        logits=logits.argmax(1).detach().cpu().numpy().tolist()
        labels=labels.detach().cpu().numpy().tolist()

        train_F1[e] += score_function(labels,logits)

    train_loss[e] /= len(train_dataloader)
    train_F1[e] /= len(train_dataloader)
        
        
    ######################    
    # 검증 모델          #
    ######################
    with torch.no_grad(): 
        model.eval()
        for data, labels in tqdm(valid_dataloader):
            # move tensors to GPU if CUDA is available
            data, labels = data.to(device), labels.to(device)
            # 순전파: compute predicted outputs by passing inputs to the model
            logits = model(data)
            # 배치 손실율 계산
            loss = criterion(logits, labels)
            # 평균 검증 손실율 
            valid_loss[e] += loss.item()
            # update training score
            logits=logits.argmax(1).detach().cpu().numpy().tolist()
            labels=labels.detach().cpu().numpy().tolist()
            valid_F1[e] += score_function(labels,logits)
            
    
    # 평균 손실율 계산산
    valid_loss[e] /= len(valid_dataloader)
    valid_F1[e] /= len(valid_dataloader)
    
    scheduler.step(valid_loss[e])    
    # 학습/검증 결과 산출 표시(loss)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        e, train_loss[e], valid_loss[e]))
    
    # 학습/검증 결과 산출 표시(정확도도)
    print('Epoch: {} \tTraining accuracy: {:.6f} \tValidation accuracy: {:.6f}'.format(
        e, train_F1[e], valid_F1[e]))
    
    # 검증 손실율이 줄어들었을 때 모델 저장장
    if valid_loss[e] <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss[e]))
        torch.save(model.state_dict(), 'swin_tiny_patch4_window7_224.pt')
        valid_loss_min = valid_loss[e]

- 여기서 순전파 (forward)랑 역전파(backward)는 강사님이 주신 자료를 보시면 이해가 빠르실 겁니다 

In [ ]:
model.load_state_dict(torch.load('swin_tiny_patch4_window7_224.pt'))

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, x_dir,transform=None):
        super().__init__()
        self.transforms = transform
        self.x_img = x_dir 

    def __len__(self):
        return len(self.x_img)

    def __getitem__(self, idx):
        x_img = self.x_img[idx]

        x_img = cv2.imread(x_img)
        x_img = cv2.cvtColor(x_img, cv2.COLOR_BGR2RGB)

        if self.transforms:
            augmented = self.transforms(image=x_img)
            x_img = augmented['image']

        return x_img

In [ ]:
means=(0.5,)
stds=(0.5,)
testtransform = albumentations.Compose([
      albumentations.Resize(224, 224),
      albumentations.augmentations.transforms.Normalize(mean=means, std=stds, p=1.0),
      albumentations.pytorch.transforms.ToTensorV2(p=1.0)
      ])

In [ ]:
batch_size=64
test_dataset = TestDataset(test,testtransform)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

- 이 위의 것들은 모델들을 로드하고 다시 결과를 산출하는 과정이라 보시면 될 듯 해요

# 결론(제일 중요)
- 우리가 해야 될 것은 뭘까요?
    - 1. 이미지 전처리(기법 활용)를 모델에 맞춰서 잘 하는 것
    - 2. effiecientb0~b7의 모델 중 좋은 모델을 찾아서
    - 3. training & validation 코드를 잘 짜고 (전체적인 과정 이해하면 더 좋아요!)
    - 4. 결과값이 나오면 어떻게 parameter를 만질지 고민, image-argumentation, ensemble, 5-fold 적극 활용!
    - 5. 최대한 다양한 모델을 만들어 보고 최적의 결과 찾기!

# 여담 

- 밑에 것들은 기본 인자값들을 설정해주는 프로그램인데 우리가 이걸 구현할 지는 모르겠어요

In [ ]:
config = { # 전역 파라미터 변수 설정정
    # Model parameters
    'model': 'efficientnet_b0',
    'batch_size': 32,
    'pretrain': True,
    
    # Optimizer parameters
    'optimizer': 'AdamW',
    'lr': 2e-4,
    'lr_t': 15,
    'lr_scheduler': 'CosineAnnealingWarmUpRestarts',
    'gamma': 0.524,
    'loss_function': 'CE_with_Lb',
    'patience': 10,
    'weight_decay': 0.002157,
    'label_smoothing': 0.8283,
    
    # Training parameters
    'epochs': 200,
    'n_fold': 5,
    'num_workers': 16,
    'text': "A",
    'device': '0,1,2,3'
    }

In [ ]:
def get_args_parser(): # 기본 인자값 부여로 보임임
    parser = argparse.ArgumentParser('PyTorch Inference', add_help=False)

    # Inference parameters
    parser.add_argument('--model_save_name', nargs='+', default='load_model', type=str)
    parser.add_argument('--model', default='efficientnet_b7', type=str)
    parser.add_argument('--batch_size', default=32, type=int)
    parser.add_argument('--pretrain', default=True, type=str2bool)
    parser.add_argument('--n_fold', default=5, type=int)
    parser.add_argument('--num_workers', default=16, type=int)
    parser.add_argument('--device', default='0,1,2,3', type=str)
    parser.add_argument('--tta', default=True, type=str2bool)
    parser.add_argument('--save_name', default='default', type=str)

마지막으로 상위권 모델 분들 보시면 정말 다양한 기법이 다 소환되니까 해석이 가능하실 때 보시면 좋아요~